<a href="https://colab.research.google.com/github/hamednasr/DDoS-Attacks/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
# Load data

data = pd.read_csv('KDDTrain.csv')


In [ ]:
data

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:
# Remove rows contain null values (in the case there are null values)
# In our data, please remove the second row on csv file before importing the file

rows, columns = np.where(pd.isnull(data))
data_filtered = data.drop(labels=rows)

print("There are %d rows having null values" %(len(data) - len(data_filtered)))

data_filtered

There are 0 rows having null values


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:
# Remove duplicate rows

data_unduplicated = data_filtered.drop_duplicates()
print("There are %d duplicated rows" %(len(data_filtered) - len(data_unduplicated)))
data_unduplicated

There are 6356 duplicated rows


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Label
0,0,146,0,0,0,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,0,0,0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune
2,0,232,8153,0,0,0,0,0,1,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,199,420,0,0,0,0,0,1,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,0,0,0,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,0,0,0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune
125968,8,105,145,0,0,0,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,2231,384,0,0,0,0,0,1,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,0,0,0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune


In [ ]:
# Define feature vectors and labels

X, y = data_unduplicated.iloc[:, :-1], data_unduplicated.iloc[:, -1]
print("Shape of X: %s" %str(X.shape), "- Type of X: %s" %str(type(X)))
print("Shape of y: %s" %str(y.shape), "- Type of y: %s" %str(type(y)))

Shape of X: (119616, 38) - Type of X: <class 'pandas.core.frame.DataFrame'>
Shape of y: (119616,) - Type of y: <class 'pandas.core.series.Series'>


In [ ]:
# Classes of data

print("The number of original classes: %d" %len(y.unique()))
y.unique()

The number of original classes: 23


array(['normal', 'neptune', 'warezclient', 'ipsweep', 'portsweep',
       'teardrop', 'nmap', 'satan', 'smurf', 'pod', 'back',
       'guess_passwd', 'ftp_write', 'multihop', 'rootkit',
       'buffer_overflow', 'imap', 'warezmaster', 'phf', 'land',
       'loadmodule', 'spy', 'perl'], dtype=object)

In [ ]:
# Transform from 23 classes to 5 main classes of Internet attacks
# normal --> normal
# back, neptune, pod, smurf, teardrop --> DOS
# ipsweep, nmap, portsweep, satan --> Probing
# ftp_write, guess_passwd, imap, multihop, phf, spy, warezclient, warezmaster --> R2L
# buffer_overflow, loadmodule, perl, rootkit --> U2R

y_array = np.array(y)
y_encoded = np.zeros((y.shape))
for i in range(len(y_array)):
    if y_array[i] in ['normal']:
        y_array[i] = 'normal'
        y_encoded[i] = 0
    elif y_array[i] in ['back', 'land', 'neptune', 'pod', 'smurf', 'teardrop']:
        y_array[i] = 'DOS'
        y_encoded[i] = 1
    elif y_array[i] in ['ipsweep', 'nmap', 'portsweep', 'satan']:
        y_array[i] = 'probing'
        y_encoded[i] = 2
    elif y_array[i] in ['ftp_write', 'guess_passwd', 'imap', 'multihop', 'phf', 'spy', 'warezclient', 'warezmaster']:
        y_array[i] = 'R2L'
        y_encoded[i] = 3
    elif y_array[i] in ['buffer_overflow', 'loadmodule', 'perl', 'rootkit']:
        y_array[i] = 'U2R'
        y_encoded[i] = 4
y_df = pd.Series(y_array)
y_df.unique()

array(['normal', 'DOS', 'R2L', 'probing', 'U2R'], dtype=object)

In [ ]:
# Check the number of each class

for name in y_df.unique():
    print("The number of samples of class %s: %d" %(str(name), len(X[y_array==name])))
y_encoded[:10]

The number of samples of class normal: 67292
The number of samples of class DOS: 39893
The number of samples of class R2L: 995
The number of samples of class probing: 11384
The number of samples of class U2R: 52


array([0., 1., 0., 0., 1., 1., 1., 1., 1., 1.])

In [ ]:
# Sort samples according to labels (optional)

data_new = X
data_new['Label'] = y_encoded
data_sorted = data_new.sort_values(by='Label')

In [ ]:
# Split the training set and testing set
data_array = data_sorted.to_numpy()
X_new, y_new = data_array[:, :-1], data_array[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=0)

# Feature selection
feature_selection = SelectKBest(chi2, k=10).fit(X_train, y_train)
X_train_new = feature_selection.transform(X_train)
X_test_new = feature_selection.transform(X_test)
print(feature_selection.get_support())

[ True  True  True False False False  True False False False False False
 False False False False False False False  True  True False  True False
 False False False False  True  True False False False False False  True
 False False]


In [ ]:
################################## CASE 1 ################################
##################### ---- WITHOUT NORMALIZATION STEP -----##############



In [ ]:

#### CNN Model
X_trn = torch.tensor(X_train_new, dtype=torch.float32)
X_trn = X_trn.unsqueeze(1)
X_tst = torch.tensor(X_test_new, dtype=torch.float32)
X_tst = X_tst.unsqueeze(1)
y_trn = torch.tensor(y_train, dtype=torch.int64)
y_tst = torch.tensor(y_test, dtype=torch.int64)

model = torch.nn.Sequential(
    # Your code for defining the model architecture here. Aim for 5-9 lines.
    torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=3//2),
    torch.nn.ReLU(),
    torch.nn.MaxPool1d(kernel_size=2),
    torch.nn.Flatten(),
    torch.nn.Linear(50, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=10, out_features=5),
)
batch_size = 100
num_epoch = 10

# Your code to define loss function and optimizer here. Aim for 2 lines.
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.001)

next_epoch = 1

for epoch in range(next_epoch, next_epoch+num_epoch):
    # Make an entire pass (an 'epoch') over the training data in batch_size chunks
    for i in range(0, len(X_trn), batch_size):        
        X = X_trn[i:i+batch_size]     # Slice out a mini-batch of features
        y = y_trn[i:i+batch_size]     # Slice out a mini-batch of targets

        y_pred = model(X)                   # Make predictions (final-layer activations)
        l = loss(y_pred, y)                 # Compute loss with respect to predictions
        
        model.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
        l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
        optimizer.step()                    # Use the gradients to take a step with SGD.
        
    print("Epoch %2d: final minibatch had loss: %.4f" % (epoch, l.item()))
    
print("Epoch %2d: loss on test set: %.4f" % (epoch, loss(model(X_tst), y_tst)))
next_epoch = epoch+1

accuracy_trn = accuracy_score(torch.argmax(model(X_trn), axis=1), y_trn)
print("train accuracy: %f" %accuracy_trn)

accuracy_tst = accuracy_score(torch.argmax(model(X_tst), axis=1), y_tst)
print("test accuracy: %f" %accuracy_tst)

confusion_matrix(torch.argmax(model(X_tst), axis=1), y_tst)

Epoch  1: final minibatch had loss: 0.9690
Epoch  2: final minibatch had loss: 0.9690
Epoch  3: final minibatch had loss: 0.9692
Epoch  4: final minibatch had loss: 0.9692
Epoch  5: final minibatch had loss: 0.9692
Epoch  6: final minibatch had loss: 0.9692
Epoch  7: final minibatch had loss: 0.9693
Epoch  8: final minibatch had loss: 0.9693
Epoch  9: final minibatch had loss: 0.9693
Epoch 10: final minibatch had loss: 0.9693
Epoch 10: loss on test set: 0.9698
train accuracy: 0.562753
test accuracy: 0.561821


array([[13441,  8000,  2261,   214,     8],
       [    0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0]], dtype=int64)

In [ ]:
X_trn, y_trn = X_train_new, y_train
X_tst, y_tst = X_test_new, y_test
gnb = GaussianNB()
gnb.fit(X_trn, y_trn)

accuracy_trn = accuracy_score(gnb.predict(X_trn), y_trn)
print("train accuracy: %f" %accuracy_trn)

accuracy_tst = accuracy_score(gnb.predict(X_tst), y_tst)
print("test accuracy: %f" %accuracy_tst)

train accuracy: 0.355129
test accuracy: 0.354456


In [ ]:
################################## CASE 1 ################################
##################### ---- WITH NORMALIZATION STEP -----##############

# Normalize the training set 

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
std_scaler.fit(X_train_new)
X_train_scaled = std_scaler.fit_transform(X_train_new)
X_test_scaled = std_scaler.fit_transform(X_test_new)

In [ ]:
## CNN

X_trn = torch.tensor(X_train_scaled, dtype=torch.float32)
X_trn = X_trn.unsqueeze(1)
X_tst = torch.tensor(X_test_scaled, dtype=torch.float32)
X_tst = X_tst.unsqueeze(1)
y_tst = torch.tensor(y_test, dtype=torch.int64)

model = torch.nn.Sequential(
    # Your code for defining the model architecture here. Aim for 5-9 lines.
    torch.nn.Conv1d(in_channels=1, out_channels=10, kernel_size=3, stride=1, padding=3//2),
    torch.nn.ReLU(),
    torch.nn.MaxPool1d(kernel_size=2),
    torch.nn.Flatten(),
    torch.nn.Linear(50, 10),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=10, out_features=5),
)
batch_size = 100
num_epoch = 10

# Your code to define loss function and optimizer here. Aim for 2 lines.
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=0.001)

next_epoch = 1

for epoch in range(next_epoch, next_epoch+num_epoch):
    # Make an entire pass (an 'epoch') over the training data in batch_size chunks
    for i in range(0, len(X_trn), batch_size):        
        X = X_trn[i:i+batch_size]     # Slice out a mini-batch of features
        y = y_trn[i:i+batch_size]     # Slice out a mini-batch of targets

        y_pred = model(X)                   # Make predictions (final-layer activations)
        l = loss(y_pred, y)                 # Compute loss with respect to predictions
        
        model.zero_grad()                   # Reset all gradient accumulators to zero (PyTorch thing)
        l.backward()                        # Compute gradient of loss wrt all parameters (backprop!)
        optimizer.step()                    # Use the gradients to take a step with SGD.
        
    print("Epoch %2d: final minibatch had loss: %.4f" % (epoch, l.item()))
    
print("Epoch %2d: loss on test set: %.4f" % (epoch, loss(model(X_tst), y_tst)))
next_epoch = epoch+1

accuracy_trn = accuracy_score(torch.argmax(model(X_trn), axis=1), y_trn)
print("train accuracy: %f" %accuracy_trn)

accuracy_tst = accuracy_score(torch.argmax(model(X_tst), axis=1), y_tst)
print("test accuracy: %f" %accuracy_tst)


Epoch  1: final minibatch had loss: 0.1957
Epoch  2: final minibatch had loss: 0.1986
Epoch  3: final minibatch had loss: 0.1876
Epoch  4: final minibatch had loss: 0.1893
Epoch  5: final minibatch had loss: 0.1791
Epoch  6: final minibatch had loss: 0.1770
Epoch  7: final minibatch had loss: 0.1780
Epoch  8: final minibatch had loss: 0.1847
Epoch  9: final minibatch had loss: 0.1833
Epoch 10: final minibatch had loss: 0.1951
Epoch 10: loss on test set: 0.2498
train accuracy: 0.904423
test accuracy: 0.901939


In [ ]:
## Gaussian Naive Bayesian

X_trn, y_trn = X_train_scaled, y_train
X_tst, y_tst = X_test_scaled, y_test
gnb = GaussianNB()
gnb.fit(X_trn, y_trn)

accuracy_trn = accuracy_score(gnb.predict(X_trn), y_trn)
print("train accuracy: %f" %accuracy_trn)

accuracy_tst = accuracy_score(gnb.predict(X_tst), y_tst)
print("test accuracy: %f" %accuracy_tst)

train accuracy: 0.601043
test accuracy: 0.478724
